# AI 경진대회 코딩 테스트 (B형)
제출자: 이성욱(sungwook.Lee@hyundai.com)  
날짜: '21.6/28  
문제(Type B): 5차원으로 데이터를 축소, 축소를 통해 정리된 인자들의 특성을 살펴라  

**OVEVIEW**  
1. Data Load and View  
2. Feature Selection  
3. Feature Extraciton  
4. Feature Character    

## 1. Data Load and View

In [1]:
from subprocess import check_output
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn.preprocessing import StandardScaler

print(check_output(["ls","input"]).decode('utf8'))

AI경진대회 예선(B형)_data-set.zip
input.csv
output.csv



In [2]:
input = pd.read_csv('input/input.csv', header=None)
output = pd.read_csv('input/output.csv', header=None)
output = output.rename(columns={0:'Label'})

In [3]:
# Standard Scaling: Data Normalization
stscaler= StandardScaler()
stscaler.fit(input)
input_ = stscaler.transform(input)

y = output
data = pd.DataFrame(input_)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.818106,-0.325165,-0.246214,-0.012640,-0.736274,1.205771,-1.074941,0.554851,1.166372,0.445609,1.065898,0.114682,-0.506004,0.006854,1.423487,-0.674604,0.299847,-1.193585,-0.215506,-0.523844
1,-0.051535,0.224741,1.586533,-0.806663,-1.905261,-1.428647,-1.138761,-0.189653,1.009079,1.932479,0.060460,1.766632,1.395304,-0.959874,0.196900,-0.570431,0.004655,-1.283734,-1.005080,-0.494709
2,0.615563,2.101084,-0.982146,-0.264738,1.043126,2.171876,-1.185627,0.103988,0.612144,-0.328839,0.849970,-0.459181,-1.007995,1.222779,-0.832886,0.692303,-1.271343,-1.205148,-0.509579,-0.925548
3,0.607127,-2.041983,0.181954,-0.300708,-1.254648,-2.385617,-0.814498,-0.312310,1.381110,-0.885105,-0.578846,-0.599110,0.880172,-1.801167,0.113969,0.472810,-0.432572,-0.787749,1.030029,-0.761267
4,1.416870,0.361684,0.424445,-1.209425,0.349353,-0.397125,-0.460240,0.267333,-1.291405,1.225944,1.432778,0.157923,-0.985794,0.625482,-0.446556,-2.052565,-0.418550,-0.468195,1.015300,-0.358104


## 2. Feature Selection
Output data is binanry data, i.e. it is classification problem.  
using `SelectFromModel`, features were selected that is the biggest one among others.  
Used classifier model is `RandomForestClassifier and GradientBoostingClassifier`.

### 2-1. RandomForestClassifier
- feature selection result: `features_forest`

In [4]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel

In [5]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(data,y)

model = SelectFromModel(clf, prefit=True)
feature_selection_using_randomforest = pd.DataFrame(model.transform(data))

features_forest = pd.Series(clf.feature_importances_)
features_forest = abs(features_forest).sort_values(ascending=False)

print("Selected Feature is {}".format(features_forest.index.values))

Selected Feature is [17  6 19 16  0  4  8  2 18 12 13  1 15  9  5  7  3 10 11 14]


### 2-2. GradientBoostingClassifier
- feature selection result: `features_boost`

In [6]:
clf=GradientBoostingClassifier(n_estimators=100)
clf = clf.fit(data,y)

model = SelectFromModel(clf, prefit=True)
feature_selection_using_boost = pd.DataFrame(model.transform(data))

features_boost=pd.Series(clf.feature_importances_)
features_boost=abs(features_boost).sort_values(ascending=False)

print("Selected Feature is {}".format(features_boost.index.values))

Selected Feature is [17  6 19  8  5 18  9 13  4  7  0 11  2 14 12 16 10  1 15  3]


### 2-3. 2norm select 
- using 2-norm between `features_forest` and `features_boost`, the biggest feature importance weights were selected.
- result: `selected_features`

In [7]:
import numpy as np

features_set = pd.concat([features_forest, features_boost], axis=1)
features_set=features_set.rename( columns={0:'forest', 1:'boost'} )
features_set['2norm'] = np.sqrt(features_set['forest']**2 + features_set['boost']**2)
selected_features = features_set['2norm'].sort_values(ascending=False).index[:5].values
print("Final Selected Features are {}".format(selected_features))

Final Selected Features are [17  6 19 16  0]


## 3. Feature Extraciton  
- results: `feature_extracted_data`

In [8]:
feature_extracted_data = data[selected_features]
feature_extracted_data.head()

,17,6,19,16,0
0,-1.193585,-1.074941,-0.523844,0.299847,0.818106
1,-1.283734,-1.138761,-0.494709,0.004655,-0.051535
2,-1.205148,-1.185627,-0.925548,-1.271343,0.615563
3,-0.787749,-0.814498,-0.761267,-0.432572,0.607127
4,-0.468195,-0.460240,-0.358104,-0.418550,1.416870


## 4. Feature Character 
- extracted feature만을 가지고 학습하였을 때 결과: 99% up, i.e. 잘 선택되어졌음을 확인
- 기본적인 산술 정보


In [9]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(feature_extracted_data,y)

score= clf.score(feature_extracted_data, y)
print("Accuracy(%): {}%".format(score*100))

Accuracy(%): 100.0%


In [33]:
#Standard Scaler 도메인에서의 데이터 말고, 원래 도메인에서의 데이터를 뽑아달라고 할수도 있으니,
final_original_domain_selected_data=input[selected_features]
final_original_domain_selected_data.head()

,17,6,19,16,0
0,-1.029791,-0.598511,-0.599297,0.308132,0.830360
1,-1.107534,-0.633978,-0.566364,0.012611,-0.043999
2,-1.039763,-0.660023,-1.053385,-1.264811,0.626718
3,-0.679803,-0.453772,-0.867680,-0.425105,0.618236
4,-0.404224,-0.256896,-0.411946,-0.411067,1.432372


## 끝.
- 이번 문제는 feature selection 문제가 아니었고, 차원을 줄이라는 문제였음 . 따라서, 위에 푼거는 아깝지만 삭제요,,

- `PCA` 특성: Principal Component Analysis는 대표적인 Linear 차원 축소 기법